In [3]:
import pandas as pd
from scipy import linalg,dot
import sklearn.metrics
import numpy as np

In [4]:
def transfer_from_dataframe_to_matrix(dataframe,row,col,value):
    matrix_row = dataframe[row].max()
    matrix_col = dataframe[col].max()
    matrix = np.zeros((int(matrix_row),int(matrix_col)))
    for index,row_content in dataframe.iterrows():
        matrix[int(row_content[row]) - 1][int(row_content[col]) -1] =row_content[value]
    return matrix

user_activity: user ID, item ID, Fitting Rating, Style Rating, Comments

In [5]:
user_activity = sc.textFile("/home/jinghong/Recommendation_System/rateFakeData1.txt")
user_activity_header = user_activity.take(1)[0]
user_activity_without_header = user_activity.filter(lambda l: l != user_activity_header)
user_activity_filter = user_activity_without_header.filter(lambda l: l.split(' ')[0].isdigit() and l.split(' ')[1].isdigit() and l.split(' ')[3].isdigit())
for i in user_activity_filter.take(10):print(i)
user_activity_filter_map = user_activity_filter.map(lambda l: (int(l.split(' ')[0]),int(l.split(' ')[1]),int(l.split(' ')[3])))
for i in user_activity_filter_map.take(10):print(i)

1 58 2 5 
2 75 5 1 
2 61 3 3 
2 17 5 3 
3 8 5 2 
3 22 2 1 
3 23 5 5 
3 64 3 5 
4 76 3 5 
4 33 1 1 
(1, 58, 5)
(2, 75, 1)
(2, 61, 3)
(2, 17, 3)
(3, 8, 2)
(3, 22, 1)
(3, 23, 5)
(3, 64, 5)
(4, 76, 5)
(4, 33, 1)


In [6]:
user_activity_df = user_activity_filter_map.toDF(schema=['User_ID','Item_ID','Style_Rating']).toPandas()
userId = "User_ID"
itemId = "Item_ID"
ratingId = "Style_Rating"
user_rating_matrix = transfer_from_dataframe_to_matrix(user_activity_df,userId,itemId,ratingId)

Edit feature here: lambda l:

In [7]:
item_RDD = sc.textFile("file:///home/jinghong/Recommendation_System/items.csv")
item_RDD_map = item_RDD.map(lambda l: (l.split(',')[0],l.split(',')[1],l.split(',')[2],l.split(',')[7]))
for i in item_RDD.take(10):print(i)

1,Yes,4,2.21,39.00,A&I Closeout,https://upload.wikimedia.org/wikipedia/commons/a/ac/No_image_available.svg,No,\N,No,4,50,70,2018-02-28 19:59:49.418,2018-02-28 19:59:49.418,1,1,2,6,189,2,2,\N,2,17,\N,\N,\N,\N,\N,\N,\N,\N,2,5,\N,\N
4,Yes,4,2.21,55.00,A&I close out,https://upload.wikimedia.org/wikipedia/commons/a/ac/No_image_available.svg,No,\N,Yes,4,50,110,2018-02-28 20:07:34.484,2018-02-28 20:07:34.484,1,1,2,6,224,2,2,\N,2,14,3,\N,\N,\N,\N,\N,3,\N,\N,\N,\N,\N
7,Yes,2,2.21,90.00,A&I close out,https://upload.wikimedia.org/wikipedia/commons/a/ac/No_image_available.svg,No,\N,Yes,3,50,110,2018-02-28 20:34:43.272,2018-02-28 20:34:43.272,1,2,2,8,224,2,2,\N,2,14,6,\N,\N,\N,\N,\N,1,\N,\N,\N,\N,\N
8,Yes,4,2.21,78.00,A&I Closeout,https://upload.wikimedia.org/wikipedia/commons/a/ac/No_image_available.svg,No,\N,No,4,40,70,2018-02-28 20:49:25.092,2018-04-03 18:31:43.402,1,1,1,6,189,2,2,\N,2,15,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,BP10001
9,Yes,3,2.21,-1.00,A&I Closeout,https://upload.wikimedia.org/wikipe

In [8]:
item_RDD_map = item_RDD_map.map(lambda l: (int(l[0]), 1 if l[1] == 'Yes' else -1, int(l[2]), 1 if l[3] == 'Yes' else -1))
for i in item_RDD_map.take(10):print(i)

(1, 1, 4, -1)
(4, 1, 4, -1)
(7, 1, 2, -1)
(8, 1, 4, -1)
(9, 1, 3, -1)
(10, 1, 4, -1)
(11, 1, 4, -1)
(13, -1, 4, -1)
(14, -1, 3, -1)
(15, -1, 2, -1)


In [9]:
item_DF = item_RDD_map.toDF(schema=['Item_ID','Ad_waist','Durability','Is_organic']).toPandas()
item_number =item_DF['Item_ID'].max() #Item ID
feature_number = item_DF.shape[1]

In [10]:
zero_matrix = np.zeros((user_rating_matrix.shape[0],item_number - user_rating_matrix.shape[1]),dtype = user_rating_matrix.dtype)
user_rating_matrix_total = np.concatenate((user_rating_matrix,zero_matrix),axis = 1)

In [11]:
item_matrix = np.zeros((item_number,feature_number))
for index,row_content in item_DF.iterrows():
    item_matrix[row_content['Item_ID'] - 1] = row_content

In [12]:
print(user_rating_matrix_total.shape)
print(item_matrix.shape)

(10000, 262)
(262, 4)


In [13]:
userprof = dot(user_rating_matrix_total,item_matrix) / linalg.norm(user_rating_matrix_total) / linalg.norm(item_matrix)
userprof

array([[ 2.10738234e-04,  3.63341782e-06,  1.81670891e-05,
        -3.63341782e-06],
       [ 2.24545221e-04,  3.63341782e-06,  1.67137220e-05,
        -5.08678495e-06],
       [ 3.43721326e-04,  7.26683565e-07,  4.43276974e-05,
        -9.44688634e-06],
       ...,
       [ 2.20911804e-04, -2.90673426e-06,  5.81346852e-06,
        -2.90673426e-06],
       [ 1.92571145e-04, -5.08678495e-06,  2.10738234e-05,
        -5.08678495e-06],
       [ 1.27896307e-04, -2.90673426e-06,  5.81346852e-06,
        -2.90673426e-06]])

In [14]:
similarityCalc = sklearn.metrics.pairwise.cosine_similarity(userprof,item_matrix,dense_output=True)

In [15]:
similarityCalc.shape #row: user, column: item

(10000, 262)

User ID - 1 = row number
e.g. final_pred[11] is the similarity of user_ID = 12

In [20]:
final_pred = np.where(similarityCalc > 0.6,1,0)


(10000, 262)


In [18]:
final_pred.shape

(10000, 262)

In [17]:
result = []
for column_index in range(item_number):
    if final_pred[11][column_index] == 1:
        result.append(item_RDD.filter(lambda l: int(l.split(',')[0])== column_index + 1).collect()[0])
result

['4,Yes,4,2.21,55.00,A&I close out,https://upload.wikimedia.org/wikipedia/commons/a/ac/No_image_available.svg,No,\\N,Yes,4,50,110,2018-02-28 20:07:34.484,2018-02-28 20:07:34.484,1,1,2,6,224,2,2,\\N,2,14,3,\\N,\\N,\\N,\\N,\\N,3,\\N,\\N,\\N,\\N,\\N',
 '7,Yes,2,2.21,90.00,A&I close out,https://upload.wikimedia.org/wikipedia/commons/a/ac/No_image_available.svg,No,\\N,Yes,3,50,110,2018-02-28 20:34:43.272,2018-02-28 20:34:43.272,1,2,2,8,224,2,2,\\N,2,14,6,\\N,\\N,\\N,\\N,\\N,1,\\N,\\N,\\N,\\N,\\N',
 '8,Yes,4,2.21,78.00,A&I Closeout,https://upload.wikimedia.org/wikipedia/commons/a/ac/No_image_available.svg,No,\\N,No,4,40,70,2018-02-28 20:49:25.092,2018-04-03 18:31:43.402,1,1,1,6,189,2,2,\\N,2,15,\\N,\\N,\\N,\\N,\\N,\\N,\\N,\\N,\\N,\\N,\\N,BP10001',
 '9,Yes,3,2.21,-1.00,A&I Closeout,https://upload.wikimedia.org/wikipedia/commons/a/ac/No_image_available.svg,No,\\N,No,4,40,60,2018-02-28 21:09:33.318,2018-02-28 21:09:33.318,1,1,3,6,227,2,2,\\N,2,14,2,\\N,\\N,\\N,\\N,\\N,3,\\N,\\N,\\N,\\N,\\N',
 '